# Clasificación de cyberbullying con BERT

<img src="../img/bert.png" width="700"/>

__Imagen tomada de Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). Bert: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805.__

## 1.- Conjuntos de datos
- Partición de entrenamiento, validación y prueba.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./cyberbullying_tweets.csv')   

In [3]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [4]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [5]:
# Contar elementos por clase
count = df['cyberbullying_type'].value_counts()
count

cyberbullying_type
religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df= train_test_split(df, test_size = 0.30, random_state = 123)

In [7]:
train_df['cyberbullying_type'].value_counts()

cyberbullying_type
gender                 5655
not_cyberbullying      5610
ethnicity              5601
age                    5573
religion               5557
other_cyberbullying    5388
Name: count, dtype: int64

In [8]:
val_df, test_df= train_test_split(test_df, test_size = 0.50, random_state = 123)

In [9]:
val_df['cyberbullying_type'].value_counts()

cyberbullying_type
religion               1250
other_cyberbullying    1219
age                    1198
gender                 1177
ethnicity              1164
not_cyberbullying      1146
Name: count, dtype: int64

In [10]:
test_df['cyberbullying_type'].value_counts()

cyberbullying_type
age                    1221
other_cyberbullying    1216
ethnicity              1196
religion               1191
not_cyberbullying      1189
gender                 1141
Name: count, dtype: int64

## 2.- Preprocesamiento

In [11]:
# Creamos un diccionario que mapea cada etiqueta a un número entero
labels_dict = {
    'not_cyberbullying': 0,
    'gender': 1,
    'religion': 2,
    'other_cyberbullying': 3,
    'age': 4,
    'ethnicity': 5
}

# Usamos la función map() para reemplazar cada etiqueta con su valor entero correspondiente
train_df['cyberbullying_type'] = train_df['cyberbullying_type'].replace(labels_dict)
val_df['cyberbullying_type'] = val_df['cyberbullying_type'].replace(labels_dict)
test_df['cyberbullying_type'] = test_df['cyberbullying_type'].replace(labels_dict)

train_df.head()

,tweet_text,cyberbullying_type
26651,@AntonSirius @erinspice @prpltnkr @ChiefElk oh...,3
4820,"Ladies ""Bedroom Bully"" The Mix Cd By @GappyRan...",0
1847,RT @_bobbidana: Never thought I'd say this but...,0
20653,She is intellectual terrorists and world suffe...,2
19195,You saudias are not friends of Muslim idiots c...,2


- Verifica cadenas vacias.

In [12]:
train_df['tweet_text'].isna().sum()

0

- Elimina puntuación y convierte a minúsculas.
- Se utiliza el método __str.translate()__ para eliminar todos los caracteres de puntuación mediante una tabla de traducción creada con el método __str.maketrans__. La constante string.punctuation contiene todos los caracteres de puntuación ASCII, que se eliminan de los valores en la columna.

In [13]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
train_df['tweet_text'] = train_df['tweet_text'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
train_df.head()

,tweet_text,cyberbullying_type
26651,antonsirius erinspice prpltnkr chiefelk oh dea...,3
4820,ladies bedroom bully the mix cd by gappyranks ...,0
1847,rt bobbidana never thought id say this but i h...,0
20653,she is intellectual terrorists and world suffe...,2
19195,you saudias are not friends of muslim idiots c...,2


In [15]:
val_df['tweet_text'] = val_df['tweet_text'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
val_df.head()

,tweet_text,cyberbullying_type
21718,but you idiot tagged and more also im a muslim,2
45048,sexylala thats a dumb nigger never fuck lala h...,5
40415,i never hear hispanics calling each other brow...,5
40212,lennybanx nigger read it again such a dumb ass...,5
35187,she was a mean girl in high school she and mim...,4


In [16]:
test_df['tweet_text'] = test_df['tweet_text'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
test_df.head()

,tweet_text,cyberbullying_type
21235,dankmtl yeap as a little propaganda nazi for t...,2
28344,lyndseyboo wow leave lyndsey phone alone bully...,3
31648,im not a man only know how to bully people,3
21779,urgedharry nyazpolitics greenlinerzjm if you a...,2
44850,calling people dumb isnt nice rt tayyoung fuc...,5


## 3.- Pipeline

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

X_train = train_df['tweet_text'].values
y_train = train_df['cyberbullying_type'].values

X_val = val_df['tweet_text'].values
y_val = val_df['cyberbullying_type'].values

X_test = test_df['tweet_text'].values
y_test = test_df['cyberbullying_type'].values

In [18]:
len(X_train), len(X_val), len(X_test)

(33384, 7154, 7154)

In [19]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
raw_val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
raw_test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [20]:
# Define a function to convert the label to a one-hot encoding
def convert_label_to_one_hot(text, label):
    one_hot_label = tf.one_hot(label, 6)
    return text, one_hot_label

# Apply the function to the dataset using map()
raw_train_ds = raw_train_ds.map(convert_label_to_one_hot)
raw_val_ds = raw_val_ds.map(convert_label_to_one_hot)
raw_test_ds = raw_test_ds.map(convert_label_to_one_hot)

batch_size = 32
train_ds = raw_train_ds.shuffle(40538).batch(batch_size)
val_ds = raw_val_ds.batch(batch_size)
test_ds = raw_test_ds.batch(batch_size)

In [21]:
for test_text, test_target in train_ds.take(1):
    print(test_text[0], test_target[0])

tf.Tensor(b'i was bullied in jr high i was embarrassed that my mom called a meeting with the principal and the girls who bullied me the aftermath neither of them talked to me again and i lost all my other friends i didn\xe2\x80\x99t have a friend until junior year of high school i survived', shape=(), dtype=string) tf.Tensor([0. 0. 0. 0. 1. 0.], shape=(6,), dtype=float32)


## 3.- Modelo

In [22]:
bert_model_path = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_preprocess_path = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [23]:
import tensorflow_hub as hub

### Capa de preprocesamiento

In [24]:
bert_preprocess_model = hub.KerasLayer(bert_preprocess_path)

In [25]:
preprocess_output = bert_preprocess_model(test_text)
preprocess_output

{'input_type_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[  101,  1045,  2001, ...,     0,     0,     0],
        [  101, 28072,  1997, ...,     0,     0,     0],
        [  101, 22135,  2378, ...,     0,     0,     0],
        ...,
        [  101, 27357,  1998, ...,     0,     0,     0],
        [  101,  1045,  2001, ...,     0,     0,     0],
        [  101,  2613,  3489, ...,     0,     0,

### BERT

In [26]:
bert_model = hub.KerasLayer(bert_model_path)
bert_model(preprocess_output)['pooled_output']

<tf.Tensor: shape=(32, 512), dtype=float32, numpy=
array([[ 0.6754461 ,  0.87300223, -0.07938088, ..., -0.23297484,
        -0.7599684 , -0.56674814],
       [ 0.9940843 ,  0.9376225 ,  0.09532023, ...,  0.15553617,
        -0.3437756 , -0.89018375],
       [ 0.9456782 ,  0.99337417,  0.10556526, ...,  0.11636379,
        -0.35791665, -0.9988588 ],
       ...,
       [ 0.91880494,  0.99298656, -0.23759362, ...,  0.10950278,
        -0.3110514 , -0.973622  ],
       [ 0.9930261 ,  0.97283703, -0.08376586, ..., -0.13618639,
        -0.45099175, -0.716974  ],
       [ 0.99551946,  0.9802226 , -0.14354205, ...,  0.23630267,
        -0.08465983, -0.92807204]], dtype=float32)>

### Modelo
- Se agrega una capa final a BERT para adaptarlo al dataset de Cyberbullying (6 clases).

In [27]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocess_text = bert_preprocess_model(text_input)
bert_output = bert_model(preprocess_text)['pooled_output']
x = tf.keras.layers.Dense(128, activation='relu')(bert_output)
output = tf.keras.layers.Dense(6)(x)
small_bert = tf.keras.Model(text_input, output)

- Prueba de las salidas de BERT.

In [28]:
small_bert(test_text)

<tf.Tensor: shape=(32, 6), dtype=float32, numpy=
array([[-1.2818291 , -0.57609606,  0.07056177, -0.3268513 ,  1.3554054 ,
        -1.5611196 ],
       [-1.079848  , -1.524926  ,  0.65202874, -0.17657088,  0.9400205 ,
        -1.3174303 ],
       [-1.0043535 , -1.6620826 ,  0.41781086, -0.3135606 ,  0.11077794,
        -2.0184412 ],
       [-0.7699677 , -1.1490512 ,  0.80352944,  0.3766127 , -0.2546184 ,
        -0.5714548 ],
       [-1.3192791 , -1.3638387 ,  0.3390544 ,  0.33671215,  1.1618032 ,
        -1.7866253 ],
       [-0.82447857, -1.3321295 ,  0.4555624 , -0.4780364 ,  0.8352588 ,
        -1.6355028 ],
       [-0.52484214, -1.193274  ,  0.71281207, -0.47896132,  0.752033  ,
        -0.74088734],
       [-0.8660503 , -1.0314274 ,  0.7957493 , -0.29176772,  1.0689174 ,
        -1.4805555 ],
       [-0.5282816 , -0.8882779 ,  0.7064259 ,  0.22719693,  0.5840322 ,
        -1.4146801 ],
       [-1.398814  , -1.507017  ,  0.20048171,  0.07269353,  1.0634654 ,
        -1.5260015 ],
 

- Muestra el tamaño del modelo y los parámetros entrenables.

In [29]:
small_bert.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

## 4.- Entrenamiento

In [30]:
lr = 0.0001
opt = tf.keras.optimizers.Adam(learning_rate=lr)

In [31]:
small_bert.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    optimizer=opt,
                    metrics=['accuracy'])

In [32]:
epochs = 5
history = small_bert.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/5
1044/1044 [==============================] - 66s 62ms/step - loss: 1.0449 - accuracy: 0.6067 - val_loss: 0.8097 - val_accuracy: 0.6947
Epoch 2/5
1044/1044 [==============================] - 64s 61ms/step - loss: 0.7767 - accuracy: 0.7037 - val_loss: 0.7284 - val_accuracy: 0.7206
Epoch 3/5
1044/1044 [==============================] - 66s 63ms/step - loss: 0.7237 - accuracy: 0.7220 - val_loss: 0.6880 - val_accuracy: 0.7354
Epoch 4/5
1044/1044 [==============================] - 65s 63ms/step - loss: 0.6927 - accuracy: 0.7334 - val_loss: 0.6743 - val_accuracy: 0.7358
Epoch 5/5
1044/1044 [==============================] - 65s 62ms/step - loss: 0.6710 - accuracy: 0.7426 - val_loss: 0.6548 - val_accuracy: 0.7457


## 5.- Evaluación

In [33]:
small_bert.evaluate(test_ds)

224/224 [==============================] - 12s 53ms/step - loss: 0.6702 - accuracy: 0.7434


[0.6701567769050598, 0.7433603405952454]

## Ejercicio
- Modifica la arquitectura y el entrenamiento para mejorar los resultados.
- Prueba diferentes versiones de BERT: https://tfhub.dev/google/collections/bert/1.